<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d5b70c492bcf4f5dec43ddba70c9d9a78b1041af11df0ac3222e3bd833be2190
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 09:21:31
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 40.03 K (0.29%)
Current PnL: -21.61 L (-14.39%)
CY Booked + Current PnL: -9.71 L (-6.47%)
-------------------
Total profit:  1.71 L
Total loss:  -23.31 L
-------------------
Total Booked + Current PnL: 17.27 L (14.03%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.51%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.80 L (63.48%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,56.0,X-LC,3.77,213624.0,9124.0,14163.0,0.04,4.46,6.63,11.39,10.0,0.64,1.54,13.81,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,95.94,48.0,M-SC,1.67,84403.0,-16374.0,16467.0,0.13,-16.25,19.51,0.09,245.0,-0.99,0.61,10.89,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.02,57.0,X-MC,1.25,211890.0,20148.0,17608.0,0.04,10.51,8.31,19.69,99.0,1.14,1.53,18.02,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.07,148829.0,12696.0,18425.0,-0.45,9.33,12.38,22.86,91.0,0.69,1.07,23.12,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,49.0,X-LC,2.85,215249.0,4883.0,23893.0,0.40,2.32,11.10,13.68,37.0,0.20,1.55,19.23,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,REPCOHOME,880.0,-57.53,62.0,H-SC,2.49,242627.0,-43878.0,301221.0,1.68,-15.31,124.15,89.82,134.0,-0.15,1.75,28.37,XY24,NTT,FINANCE
63,SHALBY,327.0,1166.35,53.0,M-SC,22.91,172141.0,-9002.0,54465.0,1.45,-4.97,31.64,25.10,235.0,-0.17,1.24,37.95,XY24,NTT,HEALTHCARE
75,TRIDENT,48.0,-10.48,45.0,M-SC,1.72,69679.0,-22576.0,48880.0,1.37,-24.47,70.15,28.51,224.0,-0.46,0.50,19.23,XR,NTT,TEXTILES
52,RAJESHEXPO,518.0,1648.74,41.0,L-SC,2.75,47724.0,-89453.0,89545.0,1.31,-65.21,187.63,0.07,267.0,-1.00,0.34,18.69,OX40N,NTT,JEWELLERY
12,BATAINDIA,2096.0,-36.87,38.0,X-SC,4.34,94371.0,-34299.0,79593.0,1.18,-26.66,84.34,35.20,219.0,-0.43,0.68,8.32,X40,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.4,856.52,37.0,H-SC,5.19,82440.0,-26280.0,105960.0,-1.58,-24.17,128.53,73.29,161.0,-0.25,0.59,10.27,XR,NTT,ENTERTAINMENT
65,SIS,528.0,2004.66,42.0,H-SC,4.00,84928.0,-26104.0,49708.0,-0.79,-23.51,58.53,21.26,156.0,-0.53,0.61,14.75,OX40N,NTT,MISC
67,STARHEALTH,761.0,17.83,63.0,H-SC,8.75,262290.0,-18833.0,151682.0,-0.61,-6.70,57.83,47.26,144.0,-0.12,1.89,40.53,XY24,NTT,INSURANCE
39,INFY,2275.0,-19.96,45.0,X-LC,7.55,312130.0,-758.0,172452.0,-0.61,-0.24,55.25,54.87,3.0,-0.00,2.25,6.39,X40,BTT,IT
15,CAMPUS,393.0,-22.54,61.0,M-SC,2.82,161255.0,-6520.0,62357.0,-0.47,-3.89,38.67,33.28,210.0,-0.10,1.16,24.98,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,17.64,150111.0,891.0,143296.0,1.10,0.60,95.46,96.62,208.0,0.01,1.08,62.45,XY25,NTT,FINANCE
59,SAIL,228.0,41.94,46.0,M-MC,12.16,223498.0,-1464.0,169121.0,-0.38,-0.65,75.67,74.53,192.0,-0.01,1.61,31.39,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,61.0,H-SC,11.93,131922.0,-10557.0,30078.0,0.60,-7.41,22.80,13.70,163.0,-0.35,0.95,55.70,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,95.94,48.0,M-SC,1.67,84403.0,-16374.0,16467.0,0.13,-16.25,19.51,0.09,245.0,-0.99,0.61,10.89,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,51.0,H-SC,1.93,221940.0,-47727.0,84071.0,0.38,-17.70,37.88,13.47,138.0,-0.57,1.60,12.85,XY24,NTT,PAINTS
17,CERA,9475.0,-22.91,45.0,H-SC,2.18,140334.0,-35569.0,77591.0,0.80,-20.22,55.29,23.89,149.0,-0.46,1.01,21.20,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.55,43.0,H-MC,7.18,101964.0,-29871.0,72099.0,-0.02,-22.66,70.71,32.03,98.0,-0.41,0.73,14.21,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.14,207625.0,4455.0,37684.0,0.45,2.19,18.15,20.74,55.0,0.12,1.50,7.86,XY25,NTT,FINANCE
31,HINDZINC,730.22,36.83,69.0,M-LC,9.52,222938.0,17862.0,94704.0,-0.14,8.71,42.48,54.89,52.0,0.19,1.61,34.90,X5K,ATH,METALS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.84,155167.0,4222.0,105234.0,0.02,2.80,67.82,72.51,53.0,0.04,1.12,8.99,XR,NTT,IT
78,UNIONBANK,163.00,-11.63,58.0,M-LC,8.79,159635.0,18795.0,25701.0,0.69,13.34,16.10,31.59,66.0,0.73,1.15,43.37,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,64.0,H-MC,13.07,195003.0,15195.0,97209.0,0.36,8.45,49.85,62.52,88.0,0.16,1.41,45.42,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.83,69.0,M-LC,9.52,222938.0,17862.0,94704.0,-0.14,8.71,42.48,54.89,52.0,0.19,1.61,34.90,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,64.0,H-MC,13.07,195003.0,15195.0,97209.0,0.36,8.45,49.85,62.52,88.0,0.16,1.41,45.42,XR,NTT,BANKS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.84,155167.0,4222.0,105234.0,0.02,2.80,67.82,72.51,53.0,0.04,1.12,8.99,XR,NTT,IT
36,INDIAMART,4810.62,-55.39,40.0,H-SC,8.07,124815.0,1479.0,130145.0,1.09,1.20,104.27,106.72,119.0,0.01,0.90,24.86,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-47.10,41.0,M-SC,4.34,89362.0,-2136.0,80890.0,0.72,-2.34,90.52,86.07,223.0,-0.03,0.64,29.77,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,5.46,141987.0,-134536.0,244601.0,0.08,-48.65,172.27,39.81,54.0,-0.55,1.02,0.08,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.02,34.0,X-MC,1.87,228010.0,-19721.0,94784.0,-0.18,-7.96,41.57,30.30,92.0,-0.21,1.64,4.27,X40,ATH,ELECTRICAL
51,QUESS,986.00,-47.11,36.0,X-SC,38.51,52363.0,-12643.0,163572.0,0.34,-19.45,312.38,232.18,198.0,-0.08,0.38,2.11,XY24,NTT,MISC
12,BATAINDIA,2096.00,-36.87,38.0,X-SC,4.34,94371.0,-34299.0,79593.0,1.18,-26.66,84.34,35.20,219.0,-0.43,0.68,8.32,X40,NTT,FOOTWEAR
34,ICICIPRULI,790.00,-23.06,40.0,X-MC,2.21,132402.0,-3386.0,46142.0,-0.37,-2.49,34.85,31.48,107.0,-0.07,0.95,9.32,X40,ATH,INSURANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.83,45.0,H-LC,0.62,156170.0,-29925.0,77398.0,0.15,-16.08,49.56,25.51,15.0,-0.39,1.13,15.09,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.02,57.0,X-MC,1.25,211890.0,20148.0,17608.0,0.04,10.51,8.31,19.69,99.0,1.14,1.53,18.02,XY25,NTT,AC
28,HAVELLS,2069.16,-8.02,34.0,X-MC,1.87,228010.0,-19721.0,94784.0,-0.18,-7.96,41.57,30.30,92.0,-0.21,1.64,4.27,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.07,148829.0,12696.0,18425.0,-0.45,9.33,12.38,22.86,91.0,0.69,1.07,23.12,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-6.82,63.0,X-LC,2.40,286861.0,21435.0,35370.0,0.37,8.08,12.33,21.40,11.0,0.61,2.07,15.26,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,5.46,141987.0,-134536.0,244601.0,0.08,-48.65,172.27,39.81,54.0,-0.55,1.02,0.08,XY24,NTT,AUTO
51,QUESS,986.00,-47.11,36.0,X-SC,38.51,52363.0,-12643.0,163572.0,0.34,-19.45,312.38,232.18,198.0,-0.08,0.38,2.11,XY24,NTT,MISC
73,TCS,4389.97,-29.97,45.0,X-LC,12.49,278672.0,-67284.0,133985.0,-0.18,-19.45,48.08,19.28,1.0,-0.50,2.01,3.02,X40,ATH,IT
82,VBL,671.64,-20.49,44.0,X-LC,10.71,286079.0,-29763.0,142439.0,0.68,-9.42,49.79,35.67,5.0,-0.21,2.06,3.13,X40N,ATH,FMCG
21,COLPAL,3767.14,-4.76,41.0,X-MC,7.93,222400.0,-40965.0,154323.0,-0.14,-15.55,69.39,43.04,84.0,-0.27,1.60,3.25,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,36.0,X-SC,38.51,52363.0,-12643.0,163572.0,0.34,-19.45,312.38,232.18,198.0,-0.08,0.38,2.11,XY24,NTT,MISC
55,RELAXO,1176.00,-39.20,44.0,X-SC,3.99,82908.0,-62252.0,138183.0,-0.23,-42.89,166.67,52.31,136.0,-0.45,0.60,10.98,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.87,38.0,X-SC,4.34,94371.0,-34299.0,79593.0,1.18,-26.66,84.34,35.20,219.0,-0.43,0.68,8.32,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.94,51.0,X-SC,1.00,108049.0,6045.0,37882.0,0.59,5.93,35.06,43.06,122.0,0.16,0.78,24.06,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.39,108450.0,-19488.0,66621.0,-0.01,-15.23,61.43,36.84,143.0,-0.29,0.78,11.03,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.49,278672.0,-67284.0,133985.0,-0.18,-19.45,48.08,19.28,1.0,-0.50,2.01,3.02,X40,ATH,IT
39,INFY,2275.00,-19.96,45.0,X-LC,7.55,312130.0,-758.0,172452.0,-0.61,-0.24,55.25,54.87,3.0,-0.00,2.25,6.39,X40,BTT,IT
41,ITC,452.00,-39.19,46.0,X-LC,2.69,196187.0,-3951.0,24837.0,0.33,-1.97,12.66,10.44,4.0,-0.16,1.41,3.48,X40,NTT,FMCG
82,VBL,671.64,-20.49,44.0,X-LC,10.71,286079.0,-29763.0,142439.0,0.68,-9.42,49.79,35.67,5.0,-0.21,2.06,3.13,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,48.0,H-LC,7.25,249024.0,-12595.0,131808.0,0.28,-4.81,52.93,45.57,7.0,-0.10,1.79,5.96,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6677.78,55.0,L-SC,6.14,74391.0,-19419.0,96039.0,1.03,-20.70,129.10,81.67,269.0,-0.20,0.54,44.06,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,51.0,L-SC,28.79,82349.0,-31200.0,71248.0,0.66,-27.48,86.52,35.27,268.0,-0.44,0.59,101.55,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,43.0,H-SC,9.47,90465.0,-7515.0,29121.0,0.15,-7.67,32.19,22.05,152.0,-0.26,0.65,31.50,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,61.0,H-SC,11.93,131922.0,-10557.0,30078.0,0.60,-7.41,22.80,13.70,163.0,-0.35,0.95,55.70,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,55.0,H-SC,10.32,145829.0,-33242.0,98974.0,0.63,-18.56,67.87,36.71,135.0,-0.34,1.05,44.69,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.64,150111.0,891.0,143296.0,1.10,0.60,95.46,96.62,208.0,0.01,1.08,62.45,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,55.0,H-SC,10.32,145829.0,-33242.0,98974.0,0.63,-18.56,67.87,36.71,135.0,-0.34,1.05,44.69,XR,NTT,BANKS
4,ANGELONE,3033.00,14.80,68.0,X-SC,9.40,202770.0,11764.0,45927.0,1.13,6.16,22.65,30.21,157.0,0.26,1.46,28.33,X40N,NTT,FINANCE
31,HINDZINC,730.22,36.83,69.0,M-LC,9.52,222938.0,17862.0,94704.0,-0.14,8.71,42.48,54.89,52.0,0.19,1.61,34.90,X5K,ATH,METALS
63,SHALBY,327.00,1166.35,53.0,M-SC,22.91,172141.0,-9002.0,54465.0,1.45,-4.97,31.64,25.10,235.0,-0.17,1.24,37.95,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.51
1,25,44.00
2,50,75.89


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.44
LC    33.71
MC    21.85
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.64
X40      14.76
XY25     12.18
XR       11.00
X40N     10.19
AR        8.20
OX40N     7.64
X200      1.83
X5K       1.61
SR        1.13
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.61
X-LC    22.55
X-MC    16.11
M-SC    12.68
M-LC     5.38
H-LC     4.75
X-SC     4.68
H-MC     3.83
M-MC     1.61
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.08,-5.02,39.63
IT,12.96,-16.89,80.12
FINANCE,12.25,-11.24,61.67
MISC,6.91,-19.86,86.28
BANKS,6.45,-11.57,70.27
PAINTS,5.57,-19.44,37.24
ELECTRICAL,5.34,-11.02,49.97
INSURANCE,3.91,-1.75,39.79
AUTO,2.81,-47.10,117.42


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3322481.0,22
XR,1382396.0,14
AR,1190967.0,9
X40,789158.0,10
X40N,614129.0,8
OX40N,585793.0,10
XY25,492916.0,8
SR,264582.0,2
X5K,94704.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3445814.0,25
M-SC,1279671.0,15
X-LC,1109550.0,13
X-MC,1091537.0,11
X-SC,531778.0,6
H-MC,348797.0,3
H-LC,281132.0,3
M-LC,263323.0,4
L-SC,256832.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197269.0      6
           XR         822355.0      7
           AR         818364.0      5
M-SC       XY24       691818.0      6
X-MC       XY24       587538.0      4
X-LC       X40        494888.0      5
           XY24       307973.0      2
X-SC       X40N       288613.0      4
H-SC       OX40N      272759.0      4
           SR         264582.0      2
X-MC       X40        214677.0      4
H-LC       AR         209206.0      2
X-LC       X40N       208125.0      3
H-MC       XY24       179489.0      1
X-MC       XY25       171931.0      2
M-MC       XY24       169121.0      1
L-SC       XR         167287.0      2
X-SC       XY24       163572.0      1
M-SC       AR         163397.0      2
           OX40N      151390.0      4
           XY25       143296.0      1
           XR         129770.0      2
X-MC       X40N       117391.0      1
M-LC       XR         105234.0      1
X-LC       XY25        98564.0      3
H-MC       XR          97209.0      1
M-LC       X5K         94704.0      1
L-SC       OX40N       89545.0      1
X-SC       X40         79593.0      1
H-MC       OX40N       72099.0      1
H-LC       X200        71926.0      1
H-SC       MH          70485.0      1
L-MC       XR          60541.0      1
L-LC       XY25        41441.0      1
M-LC       XY25        37684.0      1
           XY24        25701.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
